In [1]:
import pandas as pd
import os

In [2]:
data_dir = "/lustre/groups/ml01/workspace/ot_perturbation/models/additive_model/pbmc_new_donor/mean_model_same_cytokine"

In [3]:
dfs = []
i=0
for el in os.listdir(data_dir):
    if el.endswith(".csv"):
        df_tmp = pd.read_csv(os.path.join(data_dir, el), index_col=0)
        df_tmp = df_tmp.rename(columns={df_tmp.columns[0]: f"{df_tmp.columns[0]}_{i}"})
        if 'decoded_ood_r_squared' not in df_tmp.index:
            continue
        i+=1
        dfs.append(df_tmp)

In [4]:
len(dfs)

3000

In [4]:
len(dfs)

2880

In [5]:
df = pd.concat([el.T for el in dfs])

In [6]:
df["donor_cytokine_split-idx"] = df.index
df["donor"] = df.apply(lambda x: x["donor_cytokine_split-idx"].split("_")[0], axis=1)
df["cytokine"] = df.apply(lambda x: x["donor_cytokine_split-idx"].split("_")[1], axis=1)
df["split-idx"] = df.apply(lambda x: x["donor_cytokine_split-idx"].split("_")[2], axis=1)

/tmp/ipykernel_3441981/2560909508.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["donor_cytokine_split-idx"] = df.index
/tmp/ipykernel_3441981/2560909508.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["donor"] = df.apply(lambda x: x["donor_cytokine_split-idx"].split("_")[0], axis=1)
/tmp/ipykernel_3441981/2560909508.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.c

In [7]:
df["donor"].value_counts()

donor
Donor12    250
Donor6     250
Donor2     250
Donor3     250
Donor9     250
Donor5     250
Donor4     250
Donor11    250
Donor7     250
Donor10    250
Donor8     250
Donor1     250
Name: count, dtype: int64

In [8]:
df["cytokine"].value_counts()

cytokine
IL-32-beta    300
FasL          300
CD27L         300
OX40L         300
IFN-omega     300
IL-1Ra        300
BAFF          300
ADSF          300
M-CSF         300
IFN-gamma     300
Name: count, dtype: int64

In [9]:
df["split-idx"].value_counts()

split-idx
9     360
33    360
3     360
5     360
17    360
65    360
2     360
81    360
1     120
Name: count, dtype: int64

In [10]:
df_other = df[df["split-idx"]!="81"]
df_81 = df[df["split-idx"]=="81"]

In [11]:
df_81.shape

(360, 171)

In [12]:
df_81 = df_81.drop_duplicates("decoded_ood_r_squared")

In [13]:
df_81.shape

(120, 171)

In [14]:
df_final = pd.concat((df_other, df_81))

In [15]:
df_final["num_cytokines_in_train"].value_counts()

num_cytokines_in_train
9     360
33    360
3     360
5     360
17    360
65    360
2     360
1     120
81    120
Name: count, dtype: int64

In [16]:
df_final.to_csv(os.path.join(data_dir, "metrics_mean_model_2.csv"))